NAME: YASH PATNI

ROLL NO.: A4-71

PRACTICAL NO. 4

AIM:

(A) Write a program to validate a string using the parsing table generated in
practical 3. Consider valid & invalid string & print the stack & buffer input tracing.

Input: LL (1) parsing table

Implementation: String parsing rules using stack

Output: Each step-in string parsing and whether the input string is valid or invalid.


In [3]:
from tabulate import tabulate

terminals = ['a', 'b', 'c', 'p']
non_terminals = ['S', 'A', 'B', 'C']
starting_symbol = 'S'
productions = ["S->ABC/C", "A->a/bB/e", "B->p/e", "C->c"]
productions_dict = {nT: [] for nT in non_terminals}

for production in productions:
    nonterm_to_prod = production.split("->")
    alternatives = nonterm_to_prod[1].split("/")
    for alternative in alternatives:
        productions_dict[nonterm_to_prod[0]].append(alternative)

LL1_table = {}
conflicts = False

def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]
        for alternative in alternatives:
            first_2 = first(alternative)
            first_ |= first_2
    elif string in terminals:
        first_ = {string}
    elif string == '' or string == 'e':
        first_ = {'e'}
    else:
        first_2 = first(string[0])
        if 'e' in first_2:
            i = 1
            while 'e' in first_2:
                first_ |= (first_2 - {'e'})
                if string[i:] in terminals:
                    first_ |= {string[i:]}
                    break
                elif string[i:] == '':
                    first_ |= {'e'}
                    break
                first_2 = first(string[i:])
                first_ |= (first_2 - {'e'})
                i += 1
        else:
            first_ |= first_2
    return first_

def follow(nT):
    follow_ = set()
    prods = productions_dict.items()
    if nT == starting_symbol:
        follow_ |= {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for char in alt:
                if char == nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ |= follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if 'e' in follow_2:
                            follow_ |= (follow_2 - {'e'})
                            follow_ |= follow(nt)
                        else:
                            follow_ |= follow_2
    return follow_

FIRST = {non_terminal: set() for non_terminal in non_terminals}
FOLLOW = {non_terminal: set() for non_terminal in non_terminals}

for non_terminal in non_terminals:
    FIRST[non_terminal] |= first(non_terminal)

FOLLOW[starting_symbol] |= {'$'}
for non_terminal in non_terminals:
    FOLLOW[non_terminal] |= follow(non_terminal)

for non_terminal, alternatives in productions_dict.items():
    for alternative in alternatives:
        first_set_alt = first(alternative)
        for terminal in first_set_alt - {'e'}:
            if (non_terminal, terminal) not in LL1_table:
                LL1_table[(non_terminal, terminal)] = alternative
            else:
                print(f"Conflict at ({non_terminal}, {terminal})")
                conflicts = True

        if 'e' in first_set_alt or '' in first_set_alt:  # If epsilon is in FIRST(α)
            for terminal in FOLLOW[non_terminal]:
                if (non_terminal, terminal) not in LL1_table:
                    LL1_table[(non_terminal, terminal)] = alternative
                else:
                    print(f"Conflict at ({non_terminal}, {terminal})")
                    conflicts = True

if not conflicts:
    print("No conflicts found.")

# Parsing function
def parse_input(input_str):
    stack = [starting_symbol]
    buffer = list(input_str)
    stack_trace = []
    buffer_trace = [buffer[:]]
    accepted = False

    while stack:
        top_of_stack = stack[-1]
        if top_of_stack in terminals:
            if top_of_stack == buffer[0]:
                stack.pop()
                buffer.pop(0)
            else:
                break
        elif top_of_stack in non_terminals:
            key = (top_of_stack, buffer[0])
            if key in LL1_table:
                production = LL1_table[key]
                stack.pop()
                if production != 'e':
                    stack.extend(reversed(production))
            else:
                break
        else:
            break

        stack_trace.append(''.join(stack))
        buffer_trace.append(buffer[:])

        if not stack and not buffer:
            accepted = True

    return accepted, stack_trace, buffer_trace

# Validate strings
def validate_string(input_str):
    print("Parsing input string:", input_str)
    accepted, stack_trace, buffer_trace = parse_input(input_str)

    if accepted:
        print("String Accepted.")
    else:
        print("String Rejected.")

    print("\nStack Trace:")
    for step in stack_trace:
        print(step)

    print("\nBuffer Trace:")
    for step in buffer_trace:
        print(step)

# Test with some strings
validate_string("abpc")
validate_string("apc")



Conflict at (S, c)
Conflict at (B, p)
Parsing input string: abpc
String Rejected.

Stack Trace:
CBA
CBa
CB

Buffer Trace:
['a', 'b', 'p', 'c']
['a', 'b', 'p', 'c']
['a', 'b', 'p', 'c']
['b', 'p', 'c']
Parsing input string: apc
String Accepted.

Stack Trace:
CBA
CBa
CB
Cp
C
c


Buffer Trace:
['a', 'p', 'c']
['a', 'p', 'c']
['a', 'p', 'c']
['p', 'c']
['p', 'c']
['c']
['c']
[]
